# Define the Model Parameters
Define the initial probabilities, transition probabilities, and emission probabilities.

In [1]:
# Define the initial probabilities
initial_probabilities = {'Rainy': 0.6, 'Sunny': 0.4}

# Define the transition probabilities
transition_probabilities = {
    'Rainy': {'Rainy': 0.7, 'Sunny': 0.3},
    'Sunny': {'Rainy': 0.4, 'Sunny': 0.6}
}

# Define the emission probabilities
emission_probabilities = {
    'Rainy': {'Walk': 0.1, 'Shop': 0.4, 'Clean': 0.5},
    'Sunny': {'Walk': 0.6, 'Shop': 0.3, 'Clean': 0.1}
}

# Plot the Markov chain
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import to_agraph



# Initialize the Model
Initialize the Hidden Markov Model with the defined parameters.

In [ ]:
# Define the states and observations
states = ('Rainy', 'Sunny')
observations = ('Walk', 'Shop', 'Clean')

# Initialize the Hidden Markov Model
class HiddenMarkovModel:
    def __init__(self, states, observations, initial_probabilities, transition_probabilities, emission_probabilities):
        self.states = states
        self.observations = observations
        self.initial_probabilities = initial_probabilities
        self.transition_probabilities = transition_probabilities
        self.emission_probabilities = emission_probabilities

# Create an instance of the Hidden Markov Model
hmm = HiddenMarkovModel(states, observations, initial_probabilities, transition_probabilities, emission_probabilities)

# Implement the Forward Algorithm
Implement the forward algorithm for calculating the probability of a sequence of observations.

In [ ]:
# Implement the Forward Algorithm
def forward_algorithm(self, obs_seq):
    # Initialize the forward probabilities matrix
    forward_probs = [{}]

    # Calculate the forward probabilities for the first observation
    for state in self.states:
        forward_probs[0][state] = self.initial_probabilities[state] * self.emission_probabilities[state][obs_seq[0]]

    # Calculate the forward probabilities for the rest of the observations
    for t in range(1, len(obs_seq)):
        forward_probs.append({})
        for state in self.states:
            forward_probs[t][state] = sum(forward_probs[t-1][prev_state] * self.transition_probabilities[prev_state][state] * self.emission_probabilities[state][obs_seq[t]] for prev_state in self.states)

    # Return the forward probabilities matrix
    return forward_probs

# Add the forward algorithm to the HiddenMarkovModel class
HiddenMarkovModel.forward_algorithm = forward_algorithm

# Test the forward algorithm
obs_seq = ('Walk', 'Shop', 'Clean')
forward_probs = hmm.forward_algorithm(obs_seq)
print(forward_probs)

# Implement the Backward Algorithm
Implement the backward algorithm for calculating the probability of a sequence of observations.

In [ ]:
# Implement the Backward Algorithm
def backward_algorithm(self, obs_seq):
    # Initialize the backward probabilities matrix
    backward_probs = [{} for _ in range(len(obs_seq))]

    # Initialize the final state probabilities
    for state in self.states:
        backward_probs[-1][state] = 1

    # Calculate the backward probabilities
    for t in reversed(range(len(obs_seq)-1)):
        for state in self.states:
            backward_probs[t][state] = sum(self.transition_probabilities[state][next_state] * self.emission_probabilities[next_state][obs_seq[t+1]] * backward_probs[t+1][next_state] for next_state in self.states)

    # Return the backward probabilities matrix
    return backward_probs

# Add the backward algorithm to the HiddenMarkovModel class
HiddenMarkovModel.backward_algorithm = backward_algorithm

# Test the backward algorithm
backward_probs = hmm.backward_algorithm(obs_seq)
print(backward_probs)

In [ ]:
# Create a bar chart
fig, ax = plt.subplots()

# Initialize the bar chart with the final state probabilities
bars = plt.bar(backward_probs[-1].keys(), backward_probs[-1].values())

# Update function for the animation
def update(num, backward_probs, bars):
    # Update the heights of the bars
    for bar, state in zip(bars, backward_probs[-num].keys()):
        bar.set_height(backward_probs[-num][state])

    # Update the title of the plot
    plt.title(f'Backward Probabilities at Time {-num}')

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=len(backward_probs), fargs=[backward_probs, bars])

# Display the animation
plt.show()

# Implement the Viterbi Algorithm
Implement the Viterbi algorithm for finding the most likely sequence of hidden states.

In [ ]:
# Implement the Viterbi Algorithm
def viterbi_algorithm(self, obs_seq):
    # Initialize the Viterbi probabilities matrix
    viterbi_probs = [{}]
    # Initialize the optimal path matrix
    opt_path = {}

    # Calculate the Viterbi probabilities for the first observation
    for state in self.states:
        viterbi_probs[0][state] = self.initial_probabilities[state] * self.emission_probabilities[state][obs_seq[0]]
        opt_path[state] = [state]

    # Calculate the Viterbi probabilities for the rest of the observations
    for t in range(1, len(obs_seq)):
        viterbi_probs.append({})
        new_path = {}

        for state in self.states:
            # Select the state that maximizes the probability
            (prob, state_max) = max((viterbi_probs[t-1][prev_state] * self.transition_probabilities[prev_state][state] * self.emission_probabilities[state][obs_seq[t]], prev_state) for prev_state in self.states)
            viterbi_probs[t][state] = prob
            new_path[state] = opt_path[state_max] + [state]

        # Update the path
        opt_path = new_path

    # Select the final state that maximizes the probability
    (prob_max, state_max) = max((viterbi_probs[-1][state], state) for state in self.states)

    # Return the optimal path and the maximum probability
    return opt_path[state_max], prob_max

# Add the Viterbi algorithm to the HiddenMarkovModel class
HiddenMarkovModel.viterbi_algorithm = viterbi_algorithm

# Test the Viterbi algorithm
opt_path, prob_max = hmm.viterbi_algorithm(obs_seq)
print('The optimal path is {} with a probability of {}'.format(opt_path, prob_max))

# Implement the Baum-Welch Algorithm
Implement the Baum-Welch algorithm for training the model.

In [ ]:
# Implement the Baum-Welch Algorithm
def baum_welch_algorithm(self, obs_seq, iterations):
    # Initialize the number of states and observations
    num_states = len(self.states)
    num_obs = len(obs_seq)

    # Initialize the alpha and beta matrices
    alpha = self.forward_algorithm(obs_seq)
    beta = self.backward_algorithm(obs_seq)

    # Iterate until convergence
    for _ in range(iterations):
        # Initialize the xi and gamma matrices
        xi = [[[0.0 for _ in range(num_states)] for _ in range(num_states)] for _ in range(num_obs-1)]
        gamma = [[0.0 for _ in range(num_states)] for _ in range(num_obs)]

        # Calculate the xi and gamma matrices
        for t in range(num_obs-1):
            denom = sum(alpha[t][i] * beta[t][i] for i in self.states)
            for i in self.states:
                gamma[t][i] = (alpha[t][i] * beta[t][i]) / denom
                for j in self.states:
                    xi[t][i][j] = (alpha[t][i] * self.transition_probabilities[i][j] * self.emission_probabilities[j][obs_seq[t+1]] * beta[t+1][j]) / denom

        # Calculate the gamma matrix for the last observation
        denom = sum(alpha[-1][i] * beta[-1][i] for i in self.states)
        for i in self.states:
            gamma[-1][i] = (alpha[-1][i] * beta[-1][i]) / denom

        # Update the initial probabilities
        for i in self.states:
            self.initial_probabilities[i] = gamma[0][i]

        # Update the transition probabilities
        for i in self.states:
            for j in self.states:
                numer = sum(xi[t][i][j] for t in range(num_obs-1))
                denom = sum(gamma[t][i] for t in range(num_obs-1))
                self.transition_probabilities[i][j] = numer / denom

        # Update the emission probabilities
        for i in self.states:
            for obs in self.observations:
                numer = sum(gamma[t][i] for t in range(num_obs) if obs_seq[t] == obs)
                denom = sum(gamma[t][i] for t in range(num_obs))
                self.emission_probabilities[i][obs] = numer / denom

# Add the Baum-Welch algorithm to the HiddenMarkovModel class
HiddenMarkovModel.baum_welch_algorithm = baum_welch_algorithm

# Test the Baum-Welch algorithm
hmm.baum_welch_algorithm(obs_seq, 10)
print('The updated initial probabilities are {}'.format(hmm.initial_probabilities))
print('The updated transition probabilities are {}'.format(hmm.transition_probabilities))
print('The updated emission probabilities are {}'.format(hmm.emission_probabilities))

# Test the Model
Test the model with a sequence of observations and evaluate its performance.

In [ ]:
# Test the Model
# Define a sequence of observations to test the model
test_obs_seq = ('Walk', 'Shop', 'Clean', 'Walk', 'Walk')

# Use the Viterbi algorithm to predict the states for the test observation sequence
test_opt_path, test_prob_max = hmm.viterbi_algorithm(test_obs_seq)

# Print the predicted states and the maximum probability
print('The predicted states for the test observation sequence are {} with a maximum probability of {}'.format(test_opt_path, test_prob_max))

# Use the forward algorithm to calculate the forward probabilities for the test observation sequence
test_forward_probs = hmm.forward_algorithm(test_obs_seq)

# Print the forward probabilities
print('The forward probabilities for the test observation sequence are {}'.format(test_forward_probs))

# Use the backward algorithm to calculate the backward probabilities for the test observation sequence
test_backward_probs = hmm.backward_algorithm(test_obs_seq)

# Print the backward probabilities
print('The backward probabilities for the test observation sequence are {}'.format(test_backward_probs))